## LSTM!!!

In [2]:
#from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adadelta, Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [3]:
text = open('datasetV2twplustag').read().lower()#open('BigData').read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

('corpus length:', 896291)
('total chars:', 40)


In [4]:
with open('datasetV2twplustag') as f: #open('BigData') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
f = [x.strip().lower() for x in content] 

end = []
vri = []
for i in range(len(content)):
    end.append(content[i].index('['))
    vri.append(content[i].index(']')-content[i].index('[')+1)
    content[i] = content[i][:-1].lower()
    
sentences = content

In [5]:
maxlen = len(max(content, key=len))

X = np.zeros((sum(vri), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((sum(vri), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    k = sum(vri[:i])-i
    for j in range(1, vri[i]):
        for t, char in enumerate(sentence[:-j]):
            X[k+j-1, t+(maxlen-len(sentence[:-j])), char_indices[char]] = 1
        y[k+j-1, char_indices[sentence[-j]]] = 1

#X = np.zeros((sum(vri), maxlen, len(chars)), dtype=np.bool)
#y = np.zeros((sum(vri), len(chars)), dtype=np.bool)
#for i, sentence in enumerate(sentences):
#    for j in range(vri[i]):
#        for t, char in enumerate(sentence[:-j]):
#            X[i, t+(maxlen-len(sentence)+j), char_indices[char]] = 1
#        y[i, char_indices[sentence[-j]]] = 1
        
print X.shape

(128049, 144, 40)


In [6]:
print('Build model...')
model = Sequential()

model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...


/home/user/anaconda2/lib/python2.7/site-packages/theano/compile/function_module.py:1474: UserWarning: You are creating a TensorVariable with float64 dtype. You requested an action via the Theano flag warn_float64={ignore,warn,raise,pdb}.
  optimizer_profile = optimizer(fgraph)


In [ ]:
for iteration in range(10):
    print()
    print('-' * 80)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=256,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    

()
--------------------------------------------------------------------------------
('Iteration', 0)
Epoch 1/1

In [43]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float32')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    choices = range(len(preds))##new
    
    #probas = np.random.multinomial(1, preds, 1)
    
    return np.random.choice(choices, p=preds) #np.argmax(probas)

In [48]:
tn = 2
for diversity in [0.2, 0.5, 1.0, 1.3, 1.5]:
    print('diversity:', diversity)

    generated = ''
    sentence = content[tn][:end[tn]+1]#text[start_index: start_index + maxlen]
    generated += sentence
    generated += '['
    print 'Generating with seed:\n"' + sentence[:] +'"'
    print 'real hashtag: ' + content[tn][end[tn]:]
    #print 'generated hashtag: ['
    #sys.stdout.write(generated+'\n[')
    sys.stdout.write('generated hashtag: [')
    
    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if (next_char==']'):
            break
        
    print '\n', '\n'

('diversity:', 0.2)
Generating with seed:
"feel good feel strong happy and grateful for training session today thank you god for everything ["
real hashtag: [mufc]
generated hashtag: []


('diversity:', 0.5)
Generating with seed:
"feel good feel strong happy and grateful for training session today thank you god for everything ["
real hashtag: [mufc]
generated hashtag: []


('diversity:', 1.0)
Generating with seed:
"feel good feel strong happy and grateful for training session today thank you god for everything ["
real hashtag: [mufc]
generated hashtag: [mckdm3]


('diversity:', 1.3)
Generating with seed:
"feel good feel strong happy and grateful for training session today thank you god for everything ["
real hashtag: [mufc]
generated hashtag: []


('diversity:', 1.5)
Generating with seed:
"feel good feel strong happy and grateful for training session today thank you god for everything ["
real hashtag: [mufc]
generated hashtag: [cyfugxh4]




In [10]:
model.save('modelKsmall168')

In [ ]:
print len(vri)
print sum(vri)

In [ ]:
print max(content, key=len)

In [ ]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
